# Preguntas sobre el ejemplo de clasificación de imágenes con PyTorch y MLP

## Dataset y procesamiento

- ¿Por qué es necesario redimensionar las imágenes a un tamaño fijo para una MLP?

Una MLP tiene como entrada un vector, el cual va a tener una cantidad de elementos dada por el ancho de la imagen multiplicado por la altura y por la profundidad de color. Si cambia el tamaño de la imagen, cambia la cantidad de elementos del vector, y la MLP no funciona. Hay que redimensionar las imágenes a un tamaño, para que todos los vectores tengan la misma cantidad de elementos.


- ¿Qué ventajas ofrece Albumentations frente a otras librerías de transformación como `torchvision.transforms`?
Para tareas complejas, Albumentations es superior en varios aspectos:

  -1.Velocidad: Albumentations es significativamente mas rapida

  -2.Transformaciones mas avanzadas: Albumentations tiene tecnicas mas modernas de aumento de datos que no estan en torchvision.

  -3.Soporte robusto para segmentacion, deteccion de objetos y clasificacion: Permite aplicar transformaciones a imagenes, mascaras de segmentacion.

  -4.Transformaciones combinadas y control de probabilidad: Es posible definir pipelines complejos con transformaciones anidadas y aplicar cada una con una probabilidad personalizada

  -5.Facilidad de uso y claridad: La sintaxis es muy clara y parecida a una receta.
  
  -6.Compatibilidad: funciona perfectamente con PyTotch, TensorFlow, Keras y otros.

- ¿Qué hace `A.Normalize()`? ¿Por qué es importante antes de entrenar una red?

A.Normalize() normaliza los valores de los píxeles de una imagen para que tengan una distribución más adecuada para entrenar redes neuronales. Toma el píxel, le resta la media del dataset, y divide por el desvío estándar del dataset (por canal). Es importante normalizar porque mejora el aprendizaje (si los valores están en rangos desiguales o muy grandes, los gradientes se desbalancean, y el entrenamiento es más lento o inestable), acelera la convergencia (Si los datos tienen media ≈ 0 y desviación ≈ 1,el optimizador (SGD, Adam, etc.) se comporta de forma más eficiente ), y evita problemas numéricos (evita que activaciones o gradientes exploten o se hagan demasiado pequeños (exploding/vanishing gradients)).

- ¿Por qué convertimos las imágenes a `ToTensorV2()` al final de la pipeline?

ToTensorV2() convierte la imagen a formato tensor, que es el que usan los modelos de PyTorch. La mayoría de las transformaciones de Albumentations (como Resize, Normalize, HorizontalFlip, etc.) operan sobre imágenes tipo numpy.ndarray. Los modelos de PyTorch esperan tensores (torch.Tensor) como entrada, no arrays de NumPy ni imágenes tipo PIL.
Por eso, ToTensorV2() es el paso final: toma una imagen numpy y la convierte en un tensor PyTorch.


## 2. Arquitectura del Modelo

- ¿Por qué usamos una red MLP en lugar de una CNN aquí? ¿Qué limitaciones tiene?

El motivo de la utilización de de una red MLP es que las imágenes fueron preprocesadas y aplanadas a vectores de entrada, permitiendo entrenar un modelo simple sin necesidad de arquitecturas convolucionales. Eso facilita el prototipado y reduce la complejidad computacional.
Por otro lado, la elección de una red MLP implica perder la información espacial de las imágenes, lo que limita la capacidad del modelo para patrones complejos.


- ¿Qué hace la capa `Flatten()` al principio de la red?

La capa Flatten() al principio de una red tiene el objetivo de convertir una imagen (o tensor multi-dimensional) en un vector unidimensional, es decir, aplana la entrada. Esto es necesario porque las capas lineales (Linear) de la MLP solo aceptan vectores de entrada, no matrices ni tensores 3D.



- ¿Qué función de activación se usó? ¿Por qué no usamos `Sigmoid` o `Tanh`?

Se usa ReLu como función de activación, por ser más rápida de calcular, por no saturarse fácilmente (evita gradientes muy pequeños), y por introducir no linealidad de forma simple y eficiente. Sigmoid y Tanh no se usan en capas ocultas porque se saturan (sus derivadas tienden a cero en extremos), y llevan a problemas de desvanecimiento del gradiente.

- ¿Qué parámetro del modelo deberíamos cambiar si aumentamos el tamaño de entrada de la imagen?

Si aumentamos el tamaño de entrada de la imagen, debemos cambiar el numero de neuronas en la capa de entrada del MLP, ya que este numero debe coincidir con la cantidad total de píxeles (features) de la imagen aplanada.

## 3. Entrenamiento y Optimización

- ¿Qué hace `optimizer.zero_grad()`?

En PyTorch, los gradientes se acumulan por defecto en cada paso (.backward()).
Por eso, antes de calcular los nuevos gradientes, debemos limpiar los anteriores con: optimizer.zero_grad(). Esto asegura que los gradientes de una iteración no se sumen a los de la anterior, lo cual causaría actualizaciones incorrectas.


- ¿Por qué usamos `CrossEntropyLoss()` en este caso?

CrossEntropyLoss() se usa en problemas de clasificación multiclase, como este, donde el modelo debe predecir una clase entre varias posibles.

Esta función combina:
- LogSoftmax, que convierte las salidas del modelo en probabilidades.
- Negative Log Likelihood (NLL), que penaliza si la probabilidad predicha para la clase verdadera es baja.

Es la elección estándar para clasificación con targets enteros (no one-hot).



- ¿Cómo afecta la elección del tamaño de batch (`batch_size`) al entrenamiento?

El batch_size define cuántos ejemplos se usan para calcular el gradiente en cada paso.

Afecta:

  - Velocidad: Batches grandes usan mejor la GPU y son más rápidos por paso.
  - Estabilidad: Batches grandes hacen que las actualizaciones sean más estables y suaves.
  - Generalización: Batches pequeños pueden ayudar a escapar de mínimos locales, favoreciendo la generalización.



- ¿Qué pasaría si no usamos `model.eval()` durante la validación?

'model.eval()' le dice a PyTorch que estamos en modo evaluación, no entrenamiento. Cambia el comportamiento de ciertas capas:

- Desactiva Dropout (ya no desactiva neuronas aleatoriamente).
- Usa las estadísticas fijas de BatchNorm, en lugar de las del batch actual.

Si no lo usamos, la validación será inconsistente o incorrecta, y puede parecer que el modelo rinde peor (o mejor) de lo que realmente lo hace.


## 4. Validación y Evaluación


- ¿Qué significa una accuracy del 70% en validación pero 90% en entrenamiento?

Esto indica que el modelo está sobreajustando (overfitting). Está memorizando los datos de entrenamiento y no logra generalizar bien a datos nuevos (validación).

- 90% en entrenamiento: el modelo aprendió bien los ejemplos vistos.

- 70% en validación: falla al predecir ejemplos no vistos.

Este desbalance suele sugerir falta de regularización, poca variación en los datos o un modelo demasiado complejo.

- ¿Qué otras métricas podrían ser más relevantes que accuracy en un problema real?

En muchos casos, la accuracy no refleja correctamente el rendimiento, especialmente con clases desbalanceadas. Otras métricas útiles son:

- Precision: ¿Qué porcentaje de lo que predije como clase X realmente lo era?
-Recall: ¿Qué porcentaje de los elementos de clase X fueron correctamente detectados?
-F1-score: Promedio balanceado entre precision y recall.
-AUC-ROC: Medida agregada del rendimiento para todas las clases.
-Confusion matrix: muestra aciertos y errores por clase.

En problemas médicos o de seguridad, suele priorizarse recall o F1-score.

- ¿Qué información útil nos da una matriz de confusión que no nos da la accuracy?

La matriz de confusión muestra cómo se distribuyen los errores del modelo, permitiendo:
- Ver qué clases confunde más entre sí.
- Detectar clases dominantes (que se predicen siempre) o ignoradas.
- Evaluar si hay bias en el modelo hacia ciertas clases.

La accuracy no distingue entre errores graves o triviales. La matriz sí.


- En el reporte de clasificación, ¿qué representan `precision`, `recall` y `f1-score`?

**Precision**  
$$
\text{Precision} = \frac{\text{True Positives}}{\text{True Positives} + \text{False Positives}}
$$

Qué tan confiables son las predicciones positivas.

**Recall**  
$$
\text{Recall} = \frac{\text{True Positives}}{\text{True Positives} + \text{False Negatives}}
$$

Qué tanto recupera de los casos que debería detectar.

**F1-Score**  
$$
\text{F1} = 2 \cdot \frac{\text{Precision} \cdot \text{Recall}}{\text{Precision} + \text{Recall}}
$$

Compensación entre precision y recall.

Son fundamentales cuando hay clases desbalanceadas, donde la accuracy puede engañar.


## 5. TensorBoard y Logging


- ¿Qué ventajas tiene usar TensorBoard durante el entrenamiento?

Las ventajas de usar TensorBoard durante el entrenamiento son:
Visualización de métricas en tiempo real: permite ver gráficas de la evolución de la pérdida (loss), accuracy u otras métricas a medida que avanza el entrenamiento. Facilita identificar si el modelo está aprendiendo correctamente.
Comparación entre experimentos: se pueden registrar distintos entrenamientos (con diferentes configuraciones o modelos) y compararlos visualmente para elegir cuál funciona mejor.
Detección de overfitting o underfitting: al graficar las métricas de entrenamiento y validación, es fácil detectar si el modelo se está sobreajustando (overfitting) o si todavía no aprende bien (underfitting).
Visualización de imágenes y predicciones: permite mostrar imágenes del dataset, predicciones realizadas por el modelo o mapas de activación, lo que es muy útil para tareas de clasificación o segmentación de imágenes.
Inspección de la arquitectura del modelo: facilita ver la estructura completa del modelo (sus capas y conexiones), lo que ayuda a entender mejor arquitecturas complejas o depurar errores.
Análisis de parámetros y gradientes: permite visualizar histogramas de pesos y gradientes durante el entrenamiento. Esto ayuda a detectar problemas como gradientes desapareciendo o explotando.
Mejor organización del trabajo: centraliza toda la información del experimento en un solo lugar, ordenado y visual, lo que facilita mantener un registro claro del progreso y resultados.



- ¿Qué diferencias hay entre loguear `add_scalar`, `add_image` y `add_text`?

Principalmente, add_scalar se usa para valores numéricos (métricas), add_image para visualizar imágenes (útil en clasificación o segmentación), y add_text para dejar comentarios, configuraciones o resultados como texto en TensorBoard.


- ¿Por qué es útil guardar visualmente las imágenes de validación en TensorBoard?

Guardar visualmente las imágenes de validación en TensorBoard es útil ya que permite inspeccionar visualmente el comportamiento del modelo. Es una herramienta valiosa para análisis cualitativo en tareas de clasificación o visión por computadora.


- ¿Cómo se puede comparar el desempeño de distintos experimentos en TensorBoard?

TensorBoard permite comparar el desempeño de distintos modelos registrando cada experimento en una carpeta separada. Esto facilita visualizar curvas de entrenamiento, métricas clave y resultados visuales, ayudando a tomar decisiones informadas sobre qué arquitectura, tamaño de imagen o estrategia de aumento es más efectiva.


## 6. Generalización y Transferencia


- ¿Qué cambios habría que hacer si quisiéramos aplicar este mismo modelo a un dataset con 100 clases?

En el archivo `1_Modelo Simple.ipynb`, el modelo `MLPClassifier` está definido con un parámetro `num_classes=10`, que determina el número de salidas de la última capa:

```python
nn.Linear(128, num_classes)
```

Para adaptarlo a un dataset con 100 clases, solo hay que cambiar ese parámetro al instanciar el modelo:

```python
model = MLPClassifier(input_size=64*64*3, num_classes=100)
```

Esto asegura que la última capa tenga 100 neuronas, una por clase.  



- ¿Por qué una CNN suele ser más adecuada que una MLP para clasificación de imágenes?

Las CNN (Redes Convolucionales) aprovechan la estructura espacial de las imágenes.  
Ventajas frente a una MLP:

- Detectan patrones locales como bordes y texturas mediante convoluciones.
- Usan menos parámetros gracias a la compartición de pesos.
- Generalizan mejor en tareas visuales complejas.
- Capturan jerarquías espaciales (de píxeles a formas y objetos).

En cambio, la MLP utilizada en el notebook trata a cada imagen como un vector plano (con `nn.Flatten()`), sin considerar su estructura 2D.


- ¿Qué problema podríamos tener si entrenamos este modelo con muy pocas imágenes por clase?

- Sobreajustarse rápidamente, memorizando el conjunto de entrenamiento.
- Tener baja capacidad de generalización, fallando en validación y test.
- Sufrir de desbalance en las predicciones, favoreciendo clases más frecuentes.
- No aprender suficiente variabilidad (iluminación, orientación, etc.).

Este problema puede mitigarse con data augmentation, regularización y/o modelos preentrenados.



- ¿Cómo podríamos adaptar este pipeline para imágenes en escala de grises?

Actualmente, el modelo espera imágenes con 3 canales (RGB), como se indica en:

```python
input_size = 64 * 64 * 3
```

Para usar imágenes en escala de grises:

1. Cambiar el `input_size` a `64 * 64 * 1 = 4096`.
2. Asegurarse de que las imágenes se conviertan a 1 canal en el preprocesamiento. Por ejemplo:

```python
A.ToGray(),  # convierte de RGB a escala de grises
```


## 7. Regularización


### Preguntas teóricas:


- ¿Qué es la regularización en el contexto del entrenamiento de redes neuronales?

La regularización es un conjunto de técnicas que se aplican durante el entrenamiento de una red neuronal para evitar el overfitting. Algunos tipos son L1 Regularization, L2 Regularization, Dropout, Early Stopping y Data Augmentation.


- ¿Cuál es la diferencia entre `Dropout` y regularización `L2` (weight decay)?

Dropout: Durante el entrenamiento, con una probabilidad p, pone a cero neuronas intermedias y sus conexiones. Esto fuerza a la red a no depender de rutas específicas y mejora la generalización.

L2 Regularization (weight decay): En este caso, se agrega un termino al costo:
Loss total = Loss original +  . wi2

Este resultado penaliza los pesos grandes, alentando al modelo a mantener pesos mas pequeños y estables. Por todo esto, ambas tecnicas son complementarias para reducir el overfitting. Mientras que dropout se aplica en capas densas o despues de las activaciones, L2 regularization se aplica en todas las capas con pesos. Finalmente, usar ambas estrategias juntas suele mejorar la generalizacion del modelo.



- ¿Qué es `BatchNorm` y cómo ayuda a estabilizar el entrenamiento?

Batch Normalization es una técnica que normaliza las activaciones de una capa dentro de un mini-batch durante el entrenamiento. El objetivo principal es reducir el problema de covariate shift interno, es decir, evitar que las distribuciones de las activaciones cambien constantemente a lo largo de las capas mientras el modelo aprende. Ayuda a estabilizar el entrenamiento al evitar que las activaciones exploten o desaparezcan, que la red aprenda más rápido, al reducir la sensibilidad a la inicialización de pesos, y tiene la posibilidad de reducir el overfitting (regularización).


- ¿Cómo se relaciona `BatchNorm` con la velocidad de convergencia?

Batch Normalization acelera la convergencia del entrenamiento porque estabiliza la distribución de las activaciones en cada capa durante el proceso de aprendizaje. Esto reduce el llamado internal covariate shift, que es el cambio constante en la distribución de las activaciones a medida que se actualizan los pesos. Al mantener activaciones con media cercana a cero y varianza cercana a uno, BatchNorm permite que el entrenamiento sea más estable y consistente. Esto facilita usar tasas de aprendizaje más altas, lo que a su vez hace que el modelo pueda llegar más rápido al mínimo de la función de pérdida. Además, BatchNorm ayuda a evitar oscilaciones bruscas en la optimización y mejora la propagación de gradientes en redes profundas, evitando que se atenúen o exploten, lo que también contribuye a un aprendizaje más rápido.


- ¿Puede `BatchNorm` actuar como regularizador? ¿Por qué?

Sí, BatchNorm puede actuar como regularizador porque durante el entrenamiento normaliza las activaciones usando la media y varianza calculadas en cada mini-batch. Como cada mini-batch es diferente, esto introduce una pequeña variabilidad o ruido en las activaciones. Este ruido estocástico dificulta que el modelo dependa demasiado de ciertas neuronas o patrones específicos, ayudando a prevenir que el modelo memorice el conjunto de entrenamiento (overfitting). Por lo tanto, BatchNorm ayuda a que el modelo generalice mejor al introducir un efecto similar al de la regularización.



- ¿Qué efectos visuales podrías observar en TensorBoard si hay overfitting?

Se podría observar que la Loss de entrenamiento disminuye constantemente, mientras que la Loss de validación comienza a aumentar después de cierto punto. Lo mismo pasa con la accuracy, ya que la de entrenamiento aumenta pero la de validación se estanca o incluso puede bajar. Puede observarse también una mayor variabilidad o fluctuación en las métricas de validación en comparación con las de entrenamiento.


- ¿Cómo ayuda la regularización a mejorar la generalización del modelo?

La regularización introduce restricciones durante el entrenamiento que fuerzan al modelo a aprender patrones más simples y generales. Así se evita que el modelo memorice los datos de entrenamiento y se disminuye el overfitting.
Algunos ejemplos de esto son L2 regularization, Dropout o early stopping.


### Actividades de modificación:


1. Agregar Dropout en la arquitectura MLP:
  - Insertar capas `nn.Dropout(p=0.5)` entre las capas lineales y activaciones.
  - Comparar los resultados con y sin `Dropout`.

Resuelto en el archivo 1_Modelo_Simple.ipynb

2. Agregar Batch Normalization:
   - Insertar `nn.BatchNorm1d(...)` después de cada capa `Linear` y antes de la activación:
     ```python
     self.net = nn.Sequential(
         nn.Flatten(),
         nn.Linear(in_features, 512),
         nn.BatchNorm1d(512),
         nn.ReLU(),
         nn.Dropout(0.5),
         nn.Linear(512, 256),
         nn.BatchNorm1d(256),
         nn.ReLU(),
         nn.Dropout(0.5),
         nn.Linear(256, num_classes)
     )
     ```

Resuelto en el archivo 1_Modelo_Simple.ipynb

![Diferencia entre los modelos](fotos_tensorboard/modelos_base.png)

3. Aplicar Weight Decay (L2):
   - Modificar el optimizador:
     ```python
     optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-4)
     ```

Resuelto en el archivo 1_Modelo_Simple.ipynb

![Diferencia entre los modelos con L2](fotos_tensorboard/modelos_L2.png)

4. Reducir overfitting con data augmentation:
   - Agregar transformaciones en Albumentations como `HorizontalFlip`, `BrightnessContrast`, `ShiftScaleRotate`.

   Resuelto en el archivo 1_Modelo_Simple.ipynb

![Diferencia entre los modelos con DA](fotos_tensorboard/modelos_L2_DA.png)

5. Early Stopping (opcional):
   - Implementar un criterio para detener el entrenamiento si la validación no mejora después de N épocas.

### Preguntas prácticas:

- ¿Qué efecto tuvo `BatchNorm` en la estabilidad y velocidad del entrenamiento?

Estabilidad del entrenamiento

- Sin BatchNorm, la función de pérdida fluctuaba más entre batches y requería más épocas para estabilizarse.
- Con BatchNorm, la pérdida disminuyó de forma más suave y consistente desde las primeras épocas.

Velocidad de convergencia

- Con BatchNorm, el modelo alcanzó una accuracy decente en menos épocas.
En algunas pruebas, la validación se estabilizó en 6–7 épocas, frente a 8–9 sin BatchNorm.
- Se pudo mantener una tasa de aprendizaje mayor sin afectar la estabilidad, lo que aceleró el entrenamiento.


- ¿Cambió la performance de validación al combinar `BatchNorm` con `Dropout`?

El modelo con solo Dropout tendía a oscilar más en validación (inestabilidad entre épocas). Al combinarlo con BatchNorm, se obtuvo una curva de validación más suave, con menos sobreajuste.



- ¿Qué combinación de regularizadores dio mejores resultados en tus pruebas?

La combinación de regularizadores que dio mejores resultados fue:
Dropout + BatchNorm + L2 + Data Augmentation
Esta configuración logró la mayor accuracy en validación (~68%) y mostró una pérdida de validación estable y decreciente, lo que indica buena generalización.
En comparación, el modelo base o con un solo regularizador tendía al sobreajuste o a una convergencia más lenta.

- ¿Notaste cambios en la loss de entrenamiento al usar `BatchNorm`?

- Descenso más suave y controlado de la pérdida de entrenamiento.
  - Las curvas de train/loss con BatchNorm (como mlp_batchnorm_L2_DA) muestran una pendiente estable y sin oscilaciones bruscas.
- Menor pérdida absoluta al final del entrenamiento:
  - Por ejemplo, modelos con BatchNorm llegan a train/loss entre 0.7–0.9 mientras que algunos sin BatchNorm se quedan por encima de 1.0.
- Mejor estabilidad entre batches:
  - En los modelos sin BatchNorm, la pérdida tiene picos irregulares o zigzags al principio (primeras épocas).
  - Con BatchNorm, la pérdida baja más uniformemente, lo cual es ideal para la convergencia del optimizador.



## 8. Inicialización de Parámetros

### Preguntas teóricas:

- ¿Por qué es importante la inicialización de los pesos en una red neuronal?

La inicialización de los pesos es importante porque determina cómo comienza el proceso de aprendizaje. Si los pesos se inicializan con valores inapropiados, puede haber problemas como gradientes muy pequeños o muy grandes, lo que dificulta o impide que la red aprenda correctamente. Además, si los pesos se inicializan todos iguales, las neuronas aprenderán lo mismo y el modelo no podrá capturar patrones complejos. Una buena inicialización ayuda a que el entrenamiento sea más rápido, estable y efectivo.

- ¿Qué podría ocurrir si todos los pesos se inicializan con el mismo valor?

Si todos los pesos se inicializan con el mismo valor, todas las neuronas de una misma capa aprenderán exactamente lo mismo durante el entrenamiento. Esto sucede porque, al tener los mismos pesos, reciben el mismo gradiente y actualizan sus parámetros de manera idéntica. Como resultado, la red pierde la capacidad de aprender características diferentes y la diversidad necesaria para representar patrones complejos. En definitiva, el modelo no podrá aprovechar su capacidad para aprender, afectando gravemente su desempeño.


- ¿Cuál es la diferencia entre las inicializaciones de Xavier (Glorot) y He?

La inicialización Xavier (Glorot) está diseñada para funciones de activación simétricas como tanh o sigmoid. Busca mantener la varianza de las activaciones y los gradientes aproximadamente constante a lo largo de todas las capas, evitando que se desvanezcan o exploten. Para ello, calcula los pesos usando una distribución con varianza basada en el número de neuronas de entrada y salida. Por otro lado, la inicialización He está pensada especialmente para redes que usan activaciones ReLU o sus variantes. Considera que ReLU elimina los valores negativos, por lo que necesita una varianza mayor para compensar esta pérdida. Por eso, calcula los pesos con una varianza basada solo en el número de neuronas de entrada.


- ¿Por qué en una red con ReLU suele usarse la inicialización de He?

La función de activación ReLU elimina todos los valores negativos, lo que hace que solo la mitad de las neuronas se activen en promedio. Esto reduce la varianza de las activaciones conforme avanzan las capas. La inicialización de He compensa esta reducción aumentando la varianza inicial de los pesos, lo que ayuda a mantener las activaciones y los gradientes con magnitudes adecuadas durante el entrenamiento. Así, la inicialización de He facilita que la red con ReLU aprenda de forma más eficiente y estable, evitando problemas como gradientes muy pequeños o que la señal se degrade a medida que pasa por las capas.


- ¿Qué capas de una red requieren inicialización explícita y cuáles no?

Las capas que requieren inicialización explícita de pesos son aquellas que tienen parámetros entrenables, como las capas fully connected (densas) y las capas convolucionales. Estas capas necesitan que sus pesos se inicialicen antes de comenzar el entrenamiento. En cambio, las capas que no requieren inicialización explícita porque no tienen pesos entrenables son las capas de activación (ReLU, Sigmoid, Tanh), las capas de pooling (MaxPooling, AveragePooling) y las capas de dropout. Las capas de normalización, como BatchNorm, tienen parámetros pero suelen inicializarse automáticamente.

### Actividades de modificación:


1. Agregar inicialización manual en el modelo:
   - En la clase `MLP`, agregar un método `init_weights` que inicialice cada capa:
     ```python
     def init_weights(self):
         for m in self.modules():
             if isinstance(m, nn.Linear):
                 nn.init.kaiming_normal_(m.weight)
                 nn.init.zeros_(m.bias)
     ```

  Resuelto en el archivo 1_Modelo_Simple.ipynb

2. Probar distintas estrategias de inicialización:
   - Xavier (`nn.init.xavier_uniform_`)
   - He (`nn.init.kaiming_normal_`)
   - Aleatoria uniforme (`nn.init.uniform_`)
   - Comparar la estabilidad y velocidad del entrenamiento.

- mlp_he (He / Kaiming)

Los pesos están distribuidos de forma asimétrica y más dispersa.
La forma coincide con lo esperado en inicializaciones adaptadas a ReLU, que generan varianzas mayores en capas profundas.
Mantuvo una val_loss baja y estabilidad general.

- mlp_xavier

La distribución es más simétrica y centrada, como se espera de Xavier.
Buena estabilidad también, pero convergence más lenta.
Alcanzó resultados similares en val_accuracy, pero con mayor pérdida hacia el final.

- mlp_uniform

Los pesos están más concentrados en un rango estrecho.
Esto puede dificultar que las neuronas aprendan representaciones útiles rápidamente.
Aunque logró resultados aceptables, fue la que mostró mayor variabilidad en val_accuracy.


3. Visualizar pesos en TensorBoard:
   - Agregar esta línea en la primera época para observar los histogramas:
     ```python
     for name, param in model.named_parameters():
         writer.add_histogram(name, param, epoch)
     ```

![Resultados Inicialización de Parámetros](fotos_tensorboard/inicializacion.png)

### Preguntas prácticas:

- ¿Qué diferencias notaste en la convergencia del modelo según la inicialización?

- He (nn.init.kaiming_normal_)

Fue la más rápida y estable en converger desde las primeras épocas.
La train_loss bajó de forma constante y la val_accuracy alcanzó valores competitivos (≈52%).
La distribución inicial de los pesos fue más amplia, adaptada al uso de activaciones ReLU.

- Xavier (nn.init.xavier_uniform_)

Mostró una convergencia suave y controlada, aunque ligeramente más lenta que He.
Su train_loss descendió bien, pero la val_accuracy final fue menor que la de He (~51%).
La distribución inicial fue simétrica y centrada, lo que favorece activaciones más suaves pero puede ralentizar el aprendizaje inicial con ReLU.

- Uniforme (nn.init.uniform_)

Fue la menos estable: aunque tuvo momentos con buena val_accuracy, mostró mayor variabilidad entre épocas.
La train_loss descendió más lentamente y con mayor dispersión.
Los pesos iniciales estuvieron concentrados en un rango muy estrecho, lo que podría haber limitado la capacidad inicial del modelo para aprender.


- ¿Alguna inicialización provocó inestabilidad (pérdida muy alta o NaNs)?

Ninguna inicialización provocó valores NaN o pérdidas excesivamente altas.
Sin embargo, la inicialización uniforme fue la menos estable, con mayor variabilidad en las curvas de entrenamiento y validación.
He y Xavier ofrecieron comportamientos estables y seguros, siendo He la más eficiente con activaciones ReLU.

- ¿Qué impacto tiene la inicialización sobre las métricas de validación?

La inicialización de pesos afectó significativamente el rendimiento en validación.
La estrategia He mostró el mejor comportamiento en cuanto a estabilidad y capacidad de generalización, reflejado en mejores métricas.
Xavier fue confiable pero más lenta, y la inicialización uniforme mostró mayor sensibilidad y menor robustez en validación.

- ¿Por qué `bias` se suele inicializar en cero?

Los bias se suelen inicializar en cero porque:

- No afectan la simetría en la red
  - A diferencia de los pesos (weight), inicializar los biases en cero no provoca que todas las neuronas aprendan lo mismo ni impide el flujo del gradiente.
- Simplifica el aprendizaje inicial
  - Empezar con bias en cero permite que la red aprenda solo a partir de los pesos, sin introducir offset arbitrarios al principio.
  - El modelo ajustará los bias automáticamente durante el entrenamiento.
- Evita valores innecesarios o desbalanceados
  - Inicializar bias con valores grandes o aleatorios puede introducir ruido o saturación temprana en las activaciones (especialmente en funciones como ReLU o sigmoid).